In [1]:
from typing import Final
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
import datetime

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
TOKEN: Final = os.getenv("BOT_TOKEN")
BOT_USERNAME: Final = os.getenv("BOT_USERNAME")

In [8]:
chat_history = {}
user_chat_map = {}


def save_user(mobile_number: str, chat_id: int):
    user_chat_map[mobile_number] = chat_id
    print(f"Saved: {mobile_number} -> {chat_id}")

def save_chat(chat_id: int, sender: str, message: str):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    entry = f"[{timestamp}] {sender}: {message}"

    if chat_id not in chat_history:
        chat_history[chat_id] = []
    chat_history[chat_id].append(entry)

    # Also save to file for persistence
    with open("chat_history.txt", "a", encoding="utf-8") as f:
        f.write(f"Chat {chat_id} -> {entry}\n")

In [9]:
async def start_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("Hello! Thanks for chatting with me 😃 I am a Banana!")


async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("I am a Banana bot! 🍌 Please type something so I can respond!")


async def custom_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("This is a custom command.")


In [10]:
def handle_response(text: str) -> str:
    processed: str = text.lower()
    if "hello" in processed: return "Hey there!"
    if "how are you" in processed: return "I am good 😃"
    if "i love python" in processed: return "Remember to subscribe! 🐍"
    return "I do not understand what you wrote..."


In [11]:
async def handle_message(update, context):
    message_type = update.message.chat.type
    text = update.message.text
    user = update.message.from_user.first_name
    chat_id = update.message.chat.id

    print(f'User ({chat_id}) in {message_type}: "{text}"')

    if message_type in ["group", "supergroup"]:
        if BOT_USERNAME in text:
            new_text: str = text.replace(BOT_USERNAME, "").strip()
            response: str = handle_response(new_text)
        else:
            return
    else:
        response: str = handle_response(text)
    
        # Save user message and bot response to history
    save_chat(chat_id, user, text)
    save_chat(chat_id, "Bot", response)

    print("Bot:", response)
    await update.message.reply_text(response)


In [12]:
def error(update, context):
    print(f"Update {update} caused error {context.error}")

In [13]:
print("Starting bot...")

app = Application.builder().token(TOKEN).build()

app.add_handler(CommandHandler("help", help_command))
app.add_handler(CommandHandler("custom", custom_command))

# Messages
app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

app.add_error_handler(error)

Starting bot...


In [ ]:
from typing import Final
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes

# Bot Token and Username
TOKEN: Final = ''
BOT_USERNAME: Final = ''


# ---------------- Commands ----------------
async def start_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text('Hello! Thanks for chatting with me 😃 I am a Banana!')


async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text('I am a Banana bot! 🍌 Please type something so I can respond!')


async def custom_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text('This is a custom command.')


# ---------------- Handle Responses ----------------
def handle_response(text: str) -> str:
    processed: str = text.lower()

    if 'hello' in processed:
        return "Hey there!"
    if 'how are you' in processed:
        return "I am good 😃"
    if 'i love python' in processed:   # fixed: removed leading space
        return "Remember to subscribe! 🐍"
    return 'I do not understand what you wrote...'


# ---------------- Handle Messages ----------------
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    message_type: str = update.message.chat.type
    text: str = update.message.text

    print(f'User ({update.message.chat.id}) in {message_type}: "{text}"')

    if message_type in ['group', 'supergroup']:
        if BOT_USERNAME in text:
            new_text: str = text.replace(BOT_USERNAME, "").strip()
            response: str = handle_response(new_text)
        else:
            return
    else:
        response: str = handle_response(text)

    print('Bot:', response)
    await update.message.reply_text(response)


# ---------------- Error Handler ----------------
def error(update: object, context: ContextTypes.DEFAULT_TYPE):
    print(f'Update {update} caused error {context.error}')


# ---------------- Main ----------------
if _name_ == '_main_':
    print('Starting bot...')
    app = Application.builder().token(TOKEN).build()

    # Commands
    app.add_handler(CommandHandler('start', start_command))
    app.add_handler(CommandHandler('help', help_command))
    app.add_handler(CommandHandler('custom', custom_command))

    # Messages
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    # Errors
    app.add_error_handler(error)

    # Polling
    print('Polling...')
    app.run_polling(poll_interval=3)

Polling...


RuntimeError: Cannot close a running event loop

In [ ]:
def send_telegram_message(mobile_number: str, message: str):
    """
    Send a message to a Telegram user identified by their mobile number.
    The mobile_number must already be mapped to a chat_id.
    """
    chat_id = user_chat_map.get(mobile_number)

    if not chat_id:
        raise ValueError(f"No chat_id found for mobile number {mobile_number}")

    bot.send_message(chat_id=chat_id, text=message)
    print(f"Message sent to {mobile_number} (chat_id: {chat_id})")